In [43]:
import sys

In [31]:
#this is code for classiq problem 4

#import shit ton of library stuff
from qiskit import *
from qiskit.visualization import plot_histogram

import math

dist = {
    '000': 0.04, # 0
    '001': 0.06, # 1
    '010': 0.07, # 2
    '011': 0.52, # 3
    '100': 0.09, # 4
    '101': 0.03, # 5
    '110': 0.08, # 6
    '111': 0.11, # 7
}

In [58]:
#understanding multinoulli distributions
#https://towardsdatascience.com/how-to-create-quantum-multinoulli-distributions-with-qiskit-cea9021f6cb9

def prob_to_angle(prob):
    return 2*math.asin(math.sqrt(prob))

qr = QuantumRegister(3)
qc = QuantumCircuit(qr)
    
# |000> 0
# |001> 1
# |010> 2
# |011> 3
# |100> 4
# |101> 5
# |110> 6
# |111> 7


# split into halves, work with the upper half
range_4_to_7 = list(filter (lambda item: item[0][0] == "1", dist.items())) 
sum_4_to_7 = sum(map(lambda item: item[1], range_4_to_7))

#only taking the four "bottom" states
# |100>
# |101>
# |110>
# |111>

qc.ry(prob_to_angle(sum_4_to_7),2) #rotate ONLY the "highest-level" bit

# split upper half into quarters, work with the upper quarter
range_6_to_7 = list(filter (lambda item: item[0][1] == "1", range_4_to_7))
sum_6_to_7 = sum(map(lambda item: item[1], range_6_to_7))

#working with
# |110>
# |111>

qc.cry(prob_to_angle(sum_6_to_7/sum_4_to_7),2,1) #controlling states being rotated with "highest-level" bit 

# change upper qubit
qc.mcry(prob_to_angle(dist['111']/sum_6_to_7),qr[1:],qr[0]) #multicontrol in order to do the last one 

# work with lower quarter of upper half
qc.x(1)

range_4_to_5 = list(filter (lambda item: item[0][1] == "0", range_4_to_7))
sum_4_to_5 = sum(map(lambda item: item[1], range_4_to_5))

qc.mcry(prob_to_angle(dist['101']/sum_4_to_5),qr[1:],qr[0]) #x gates cause now we want multicontrol on the ones with the 2nd bit in "0" pos
qc.x(1)

# work with lower half
range_0_to_3 = list(filter (lambda item: item[0][0] == "0", dist.items()))
sum_0_to_3 = sum(map(lambda item: item[1], range_0_to_3))
qc.x(2)

# split lower half into quarters, work with the upper quarter

range_2_to_3 = list(filter (lambda item: item[0][1] == "1", range_0_to_3))
sum_2_to_3 = sum(map(lambda item: item[1], range_2_to_3))

qc.cry(prob_to_angle(sum_2_to_3/sum_0_to_3),2,1)

# change upper qubit
qc.mcry(prob_to_angle(dist['011']/sum_2_to_3),qr[1:],qr[0])

# work with lower quarter of upper half
qc.x(1)

range_0_to_1 = list(filter (lambda item: item[0][1] == "0", range_0_to_3))
sum_0_to_1 = sum(map(lambda item: item[1], range_0_to_1))

qc.mcry(prob_to_angle(dist['001']/sum_0_to_1),qr[1:],qr[0])
qc.x(1)

qc.x(2)

results = execute(qc,Aer.get_backend('statevector_simulator')).result().get_counts()

qc.draw()
#plot_histogram(results, figsize=(7,2), color=['blue'])


┌─────────────┐              ┌───┐┌──────────────┐┌───┐┌─────────┐┌───┐»
q318_0: ┤ Ry(0.86468) ├──────────────┤ X ├┤ Ry(-0.86468) ├┤ X ├┤ Ry(π/6) ├┤ X ├»
        └─────────────┘┌────────────┐└─┬─┘└──────────────┘└─┬─┘└──┬───┬──┘└─┬─┘»
q318_1: ───────────────┤ Ry(1.7986) ├──■────────────────────■─────┤ X ├─────■──»
         ┌───────────┐ └─────┬──────┘  │                    │     └───┘     │  »
q318_2: ─┤ Ry(1.181) ├───────■─────────■────────────────────■───────────────■──»
         └───────────┘                                                         »
«        ┌──────────┐┌───┐┌────────────┐              ┌───┐┌─────────────┐┌───┐»
«q318_0: ┤ Ry(-π/6) ├┤ X ├┤ Ry(1.2191) ├──────────────┤ X ├┤ Ry(-1.2191) ├┤ X ├»
«        └──────────┘└─┬─┘└───┬───┬────┘┌────────────┐└─┬─┘└─────────────┘└─┬─┘»
«q318_1: ──────────────■──────┤ X ├─────┤ Ry(2.3605) ├──■───────────────────■──»
«                      │      ├───┤     └─────┬──────┘  │                   │  »
«q318_2: ──────────────■──────┤ X ├───────────■─────────■───────────────────■──»
«                             └───┘                                            »
«        ┌─────────────┐┌───┐┌──────────────┐┌───┐     
«q318_0: ┤ Ry(0.88608) ├┤ X ├┤ Ry(-0.88608) ├┤ X ├─────
«        └────┬───┬────┘└─┬─┘└──────────────┘└─┬─┘┌───┐
«q318_1: ─────┤ X ├───────■────────────────────■──┤ X ├
«             └───┘       │                    │  ├───┤
«q318_2: ─────────────────■────────────────────■──┤ X ├
«                                                 └───┘